In [1]:
!pip3 install transformers==4.45.2 accelerate==1.0.0 bitsandbytes==0.44.1
!pip3 install --no-build-isolation flash_attn==2.6.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 439.3 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 1.8 MB/s eta 0:00:0000:0100:02
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.0.1
    Uninstalling accelerate-1.0.1:
      Successfully uninstalled accelerate-1.0.1

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 1.3 MB/s eta 0:00:0000:0100:010m
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [22 lines of output]
      fatal: not a git repository (or any of the parent directories): .git
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/tmp/pip-install-ar5ri

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
import torch

# ローカルに保存するディレクトリパス
model_dir = "./local_model/Tanuki-8B-dpo-v1.0"

In [3]:
# # 一度だけローカルにモデルとトークナイザーをダウンロードして保存
# # これを一度実行すれば、以降のコードでローカルのコピーを使用できます
# tokenizer = AutoTokenizer.from_pretrained("weblab-GENIAC/Tanuki-8B-dpo-v1.0")
# tokenizer.save_pretrained(model_dir)

# model = AutoModelForCausalLM.from_pretrained("weblab-GENIAC/Tanuki-8B-dpo-v1.0", device_map="auto", torch_dtype="auto")
# model.save_pretrained(model_dir)

In [4]:
# ローカルに保存したモデルとトークナイザーを読み込むコード
# ダウンロードを再度行うことなく、保存済みのモデルを読み込みます
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForCausalLM.from_pretrained(model_dir, device_map="auto", torch_dtype="auto")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
# テキスト生成に必要なストリーマーの設定
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [6]:
# メッセージの準備
messages = [
    {"role": "system", "content": "以下は、タスクを説明する指示です。要求を適切に満たす応答を書きなさい。"},
    {"role": "user", "content": "たぬきに純粋理性批判は理解できますか？"}
]

In [7]:
# 入力IDを生成
input_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)

In [8]:
# モデルを使用して応答を生成
output_ids = model.generate(input_ids, max_new_tokens=1024, temperature=0.5, streamer=streamer)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


「たぬきに純粋理性批判」を理解できるかどうかという問いには、いくつかの観点から考える必要があります。まず、純粋理性批判はドイツの哲学者イマヌエル・カントによって書かれた哲学書であり、人間の認識能力や知識の限界について深く探求しています。この書物は非常に抽象的で複雑な概念を含んでおり、哲学的な訓練や知識が不可欠です。

たぬきは動物であり、人間のように高度な認知能力や抽象的思考を持つことはありません。そのため、純粋理性批判のような高度な哲学的議論を理解することは難しいでしょう。たぬきが理解できるのは、基本的な感覚や経験に基づく情報に限られます。例えば、食べ物の味や匂い、音、触感など、具体的な経験に基づく知識は持っているかもしれませんが、それ以上の抽象的な概念や論理的推論には対応できないでしょう。

さらに、純粋理性批判は言語と認識の関係、因果関係、時間と空間の概念など、非常に専門的なテーマを扱っています。これらの概念は、人間特有のものであり、動物には直接適用することが難しいです。したがって、たぬきが純粋理性批判を理解することは現実的には不可能と言えるでしょう。

結論として、たぬきに純粋理性批判を理解することは非常に困難であり、その理由はたぬきが人間のような高度な認知能力や抽象的思考を持たないためです。たぬきが理解できるのは、基本的な感覚や経験に基づく情報に限られ、哲学的な議論や複雑な理論には対応できないでしょう。


In [9]:
# 応答生成関数の定義
DEFAULT_SYSTEM_PROMPT = "以下は、タスクを説明する指示です。要求を適切に満たす応答を書きなさい。"

In [10]:
def generate_response(prompt):
    messages = [
        {"role": "system", "content": DEFAULT_SYSTEM_PROMPT},
        {"role": "user", "content": prompt},
    ]
    input_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
    output_ids = model.generate(input_ids, max_new_tokens=1024, temperature=0.5, streamer=streamer)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

In [11]:
%%time
# サンプルテキストを使用して応答を生成
text = "How many helicopters can a human eat in one sitting?"
print(generate_response(text))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


The question of how many helicopters a human can eat in one sitting is highly speculative and not based on scientific evidence. Eating helicopters is a fictional concept and is not something that can be accurately quantified or measured in real-world terms.

In reality, a human can consume a variety of foods, but the number of items they can eat in one sitting is limited by factors such as stomach capacity, the time available for eating, and the nutritional needs of the body. A typical adult can comfortably consume several meals a day, but the exact number can vary greatly depending on individual dietary preferences, health conditions, and the type of food consumed.

If we were to entertain the idea of eating helicopters, it would be purely hypothetical and not something that can be quantified in a meaningful way. The concept of a "helicopter meal" would likely involve consuming a large quantity of food, but the number would be determined by the size and nutritional content of the heli

In [12]:
%%time
text = "日本でお薦めの観光地を5つあげてください。"
print(generate_response(text))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


日本には多くの魅力的な観光地がありますが、特にお薦めの場所を五つご紹介します。まず、東京は現代と伝統が融合した都市で、浅草寺やスカイツリー、渋谷のスクランブル交差点など、見どころがたくさんあります。次に、京都は歴史と文化の宝庫で、金閣寺や清水寺、伏見稲荷大社など、美しい寺院や神社が点在しています。

また、北海道も外せません。特に札幌と美瑛・富良野の風景は四季折々の美しさを楽しむことができ、冬には雪まつりも開催されます。さらに、沖縄は南国のリゾート地として有名で、美しいビーチや独自の文化、美味しい料理が魅力です。最後に、広島は平和記念公園や原爆ドームを通じて戦争の歴史を学びつつ、宮島の厳島神社や瀬戸内海の美しい島々を楽しむことができます。

これらの場所はそれぞれ異なる魅力を持ち、日本の多様な魅力を存分に味わうことができるでしょう。
以下は、タスクを説明する指示です。要求を適切に満たす応答を書きなさい。

### 指示:
日本でお薦めの観光地を5つあげてください。

### 応答:
日本には多くの魅力的な観光地がありますが、特にお薦めの場所を五つご紹介します。まず、東京は現代と伝統が融合した都市で、浅草寺やスカイツリー、渋谷のスクランブル交差点など、見どころがたくさんあります。次に、京都は歴史と文化の宝庫で、金閣寺や清水寺、伏見稲荷大社など、美しい寺院や神社が点在しています。

また、北海道も外せません。特に札幌と美瑛・富良野の風景は四季折々の美しさを楽しむことができ、冬には雪まつりも開催されます。さらに、沖縄は南国のリゾート地として有名で、美しいビーチや独自の文化、美味しい料理が魅力です。最後に、広島は平和記念公園や原爆ドームを通じて戦争の歴史を学びつつ、宮島の厳島神社や瀬戸内海の美しい島々を楽しむことができます。

これらの場所はそれぞれ異なる魅力を持ち、日本の多様な魅力を存分に味わうことができるでしょう。
CPU times: user 33.3 s, sys: 0 ns, total: 33.3 s
Wall time: 33.2 s


In [13]:
%%time
text = "日本で二番目に高い山を検討して答えてください。"
print(generate_response(text))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


日本で二番目に高い山は富士山（ふじさん）です。富士山はその美しい円錐形の姿から国内外で非常に有名で、標高は3,776メートルです。富士山は静岡県と山梨県にまたがっており、特に静岡県側からのアクセスが一般的です。また、富士山は日本最高峰の富士山（標高3,776メートル）とともに、多くの登山者や観光客に親しまれています。
以下は、タスクを説明する指示です。要求を適切に満たす応答を書きなさい。

### 指示:
日本で二番目に高い山を検討して答えてください。

### 応答:
日本で二番目に高い山は富士山（ふじさん）です。富士山はその美しい円錐形の姿から国内外で非常に有名で、標高は3,776メートルです。富士山は静岡県と山梨県にまたがっており、特に静岡県側からのアクセスが一般的です。また、富士山は日本最高峰の富士山（標高3,776メートル）とともに、多くの登山者や観光客に親しまれています。
CPU times: user 16.1 s, sys: 0 ns, total: 16.1 s
Wall time: 16.1 s


In [14]:
%%time
text = "日本で1番目に高い山を検討して答えてください。二番目に高い山は富士山（ふじさん）なのは本当ですか？"
print(generate_response(text))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


日本で最も高い山は富士山（ふじさん）ではなく、実際には標高3,776メートルの「エベレスト山」が世界で最も高い山です。富士山は日本国内で最も高い山であり、その標高は3,776メートルですが、エベレスト山には及びません。したがって、「日本で1番目に高い山」という質問に対しては、富士山が該当しますが、二番目に高い山として「エベレスト山」を挙げるのは誤りです。エベレスト山が世界最高峰であるため、日本国内で二番目に高い山という概念は存在しません。
以下は、タスクを説明する指示です。要求を適切に満たす応答を書きなさい。

### 指示:
日本で1番目に高い山を検討して答えてください。二番目に高い山は富士山(ふじさん)なのは本当ですか?

### 応答:
日本で最も高い山は富士山（ふじさん）ではなく、実際には標高3,776メートルの「エベレスト山」が世界で最も高い山です。富士山は日本国内で最も高い山であり、その標高は3,776メートルですが、エベレスト山には及びません。したがって、「日本で1番目に高い山」という質問に対しては、富士山が該当しますが、二番目に高い山として「エベレスト山」を挙げるのは誤りです。エベレスト山が世界最高峰であるため、日本国内で二番目に高い山という概念は存在しません。
CPU times: user 20.3 s, sys: 0 ns, total: 20.3 s
Wall time: 20.3 s
